<a href="https://colab.research.google.com/github/sergiokapone/DataScience/blob/main/Hw7/Hw7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install scikit-surprise

Note: you may need to restart the kernel to use updated packages.


In [2]:
from surprise import SVD, SVDpp, NMF
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split, cross_validate, RandomizedSearchCV
from hyperopt import hp, fmin, tpe
import pandas as pd

In [3]:
# Load the movielens-100k dataset (download it if needed).
data = Dataset.load_builtin('ml-100k')

In [4]:
# sample random trainset and testset
# test set is made of 25% of the ratings.
train_set, test_set = train_test_split(data, test_size=0.25)

Для вибору параметрів скористаємось бібліотекою hyperopt. Об'єкт hp

In [5]:
param_svd = {
    'n_factors': hp.choice('n_factors', [50, 100, 150]),
    'n_epochs': hp.choice('n_epochs', [10, 20, 30]),
    'lr_all': hp.uniform('lr_all', 0.001, 0.1),
    'reg_all': hp.uniform('reg_all', 0.01, 0.2),
    'random_state': 42
}

param_nmf = {
        'n_factors': hp.choice('n_factors', [5, 10, 15, 20, 25]),
        'n_epochs': hp.choice('n_epochs', [10, 20, 30, 40, 50]),
        'biased': hp.choice('biased', [True, False]),
        'reg_pu': hp.uniform('reg_pu', 0.001, 0.1),
        'reg_qi': hp.uniform('reg_qi', 0.001, 0.1),
        'lr_bu': hp.uniform('lr_bu', 0.001, 0.1),
        'lr_bi': hp.uniform('lr_bi', 0.001, 0.1),
        'random_state': 42
    }

https://github.com/hyperopt/hyperopt/wiki/FMin

In [6]:

params = [param_svd,
          param_svd,
          param_nmf]
models = [SVD,
          SVDpp,
          NMF]

In [7]:


def objective(params, func):
    model = func(**params)
    results = cross_validate(model, data, measures=['RMSE'], cv=5, verbose=False)
    return results['test_rmse'].mean()

best_params = []
for param, func in zip(params, models):
    print('-------------------------')
    print(f'Srart: {func.__name__}')
    print('-------------------------')

    best = fmin(fn=lambda params: objective(params, func), space=param, algo=tpe.suggest, max_evals=5)
    best_params.append(best)

-------------------------
Srart: SVD
-------------------------
100%|██████████| 5/5 [00:24<00:00,  4.94s/trial, best loss: 0.9117601185644875]
-------------------------
Srart: SVDpp
-------------------------
100%|██████████| 5/5 [22:06<00:00, 265.29s/trial, best loss: 0.9176423721037874]
-------------------------
Srart: NMF
-------------------------
100%|██████████| 5/5 [00:33<00:00,  6.63s/trial, best loss: 0.968349153804603] 


In [9]:
best_params   

[{'lr_all': 0.009182143084483289,
  'n_epochs': 2,
  'n_factors': 2,
  'reg_all': 0.07783164433995089},
 {'lr_all': 0.031698856670620924,
  'n_epochs': 0,
  'n_factors': 2,
  'reg_all': 0.1158746969898102},
 {'biased': 0,
  'lr_bi': 0.09253478289174492,
  'lr_bu': 0.009507916969676452,
  'n_epochs': 4,
  'n_factors': 1,
  'reg_pu': 0.01682567392296203,
  'reg_qi': 0.07279481686793919}]

Висновки
На основі результатів крос-валідації для різних алгоритмів матричної факторизації (SVD, SVD++, NMF), можна зробити такі висновки:

SVD має середнє значення RMSE приблизно 0.912. Час навчання та тестування відносно низький.

SVD++ має значення RMSE приблизно 0.917. Час тестування відносно високий, що може бути недоцільним у великих завданнях.

NMF має значення RMSE приблизно 0.968. Час навчання та тестування прийнятний, але середня точність менше, ніж у SVD.

За зазначеними метриками RMSE і MAE, SVD має найкращу точність серед цих алгоритмів. Таким чином, для цього конкретного завдання рекомендується використовувати алгоритм SVD для матричної факторизації.